In [1]:
import os
os.chdir("/home/celieni/dir_thesis/TVB-Tests")

In [3]:
# importing libraries
import os
import scipy
import nitime 
import warnings
import subprocess
import numpy as np 
import scipy.io as sio
import scipy.stats as stat
from scipy import stats, signal
import matplotlib.pyplot as plt 
from scipy.sparse import lil_matrix
from nitime.utils import percent_change
from nitime.timeseries import TimeSeries
from rsHRF import spm_dep, processing, canon, sFIR, parameters, basis_functions, utils
from nitime.analysis import CorrelationAnalyzer, CoherenceAnalyzer

warnings.filterwarnings("ignore")

def executeC(): 
    # store the return code of the c program(return 0) 
    # and display the output 
    s = subprocess.check_call("gcc main.c -lpthread -lm -lgsl -lgslcblas -o tvbii; ./tvbii param_set weights tract_lengths ROIts_retrievedHRF 1", shell = True) 

def get_path(group_name, subject_num, test_num):
    # gets the path for the directory corresponding to each combination of group, subject and test
    if subject_num < 10: return "./Data/" + group_name + "0" + str(subject_num) + "/"
    else: return "./Data/" + group_name + str(subject_num) + "/"

def getHRF(group_id, subject_id):
    # retrieves the HRF from the empirical fMRI time-series
    path = get_path(group_id, subject_id, "T1")
    mat = scipy.io.loadmat(path + "FC.mat")
    # parameters required for HRF retrieval
    para = {}
    para['estimation'] = 'canon2dd'
    para['passband'] = [0.01, 0.08]
    para['TR'] = mat["TR"][0][0]
    para['T'] = 3
    para['T0'] = 1
    para['AR_lag'] = 1
    para['TD_DD'] = 2
    para['localK'] = 2
    para['thr'] = 1
    para['len'] = 25
    para['min_onset_search'] = 4
    para['max_onset_search'] = 8
    para['dt'] = para['TR'] / para['T']
    para['lag'] = np.arange(np.fix(para['min_onset_search'] / para['dt']),
                                np.fix(para['max_onset_search'] / para['dt']) + 1,
                                dtype='int')
    if subject_id < 10: bold_sig = mat[group_id + "0" + str(subject_id) + "T1_ROIts_DK68"]
    else: bold_sig = mat[group_id + str(subject_id) + "T1_ROIts_DK68"]
    bold_sig = stats.zscore(bold_sig, ddof=1)
    bold_sig = np.nan_to_num(bold_sig) # replace nan with 0 and inf with large finite numbers
    bf = basis_functions.basis_functions.get_basis_function(bold_sig.shape, para)
    beta_hrf, event_bold = utils.hrf_estimation.compute_hrf(bold_sig, para, [], 1, bf)
    hrfa = np.dot(bf, beta_hrf[np.arange(0, bf.shape[1]), :])

    np.savetxt('./C_Input/ROIts_retrievedHRF.txt', hrfa.T, delimiter=' ')
    # returns the length of the retrieved HRF and the BOLD Repetition Time
    return hrfa.shape[0], para['TR']

def make_input(group_id, subject_id):
    # does all the book-keeping with respect to arranging and channeling the input files
    try : os.remove("./C_Input/weights.txt")
    except : pass
    try: os.remove("./C_Input/tract_lengths.txt")
    except: pass
    try: os.remove("./C_Input/ROIts_retrievedHRF.txt")
    except: pass
    path = get_path(group_id, subject_id, "T1")
    np.savetxt('./C_Input/weights.txt', np.loadtxt(path + "weights.txt"), delimiter=' ')
    np.savetxt('./C_Input/tract_lengths.txt', np.loadtxt(path + "tract_lengths.txt"), delimiter=' ')
    hrf_len, TR = getHRF(group_id, subject_id)
    f = open("./C_Input/param_set.txt", "r")
    for line in f:
        temp = line.split()
        break
    f.close()
    # making relavant changes to the parameters file used for simulation
    temp[6] = str(int(TR * 1000 * 200))
    temp[7] = str(int(TR * 1000))
    temp[10] = str(hrf_len)
    f = open("./C_Input/param_set.txt", "w") 
    for each in temp:
        f.write(each)
        f.write(" ") 
    f.close()

def getCorrelation(group_id, subject_id):
    # obtains the correlation between empirical functional connectivity and simulated functional connectivity
    input_path_sim = "fMRI.txt"
    input_path_em = get_path(group_id, subject_id, "T1") + "FC.mat"
    em_mat = scipy.io.loadmat(input_path_em)
    em_fc_matrix = em_mat["FC_cc_DK68"]
    sampling_interval = em_mat["TR"][0][0]
    uidx = np.triu_indices(68, 1)
    em_fc_z = np.arctanh(em_fc_matrix)
    em_fc = em_fc_z[uidx]
    tsr = np.loadtxt(input_path_sim)
    T = TimeSeries(tsr, sampling_interval=sampling_interval)
    C = CorrelationAnalyzer(T)
    sim_fc = np.arctanh(C.corrcoef)[uidx]
    sim_fc = np.nan_to_num(sim_fc)
    pearson_corr, _ = stat.pearsonr(sim_fc, em_fc)
    return pearson_corr

def alterGlobalCoupling(G):
    # alters the global coupling value for each iteration of parameter space exploration
    f = open("./C_Input/param_set.txt", "r")
    for line in f:
        temp = line.split()
        break
    f.close()
    temp[1] = str(G) 
    f = open("./C_Input/param_set.txt", "w") 
    for each in temp:
        f.write(each)
        f.write(" ") 
    f.close()

if __name__== "__main__":
    # Driver function 
    Subjects = {}
    Subjects["CON"] = [1, 2]  #[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    Subjects["PAT"] = [1, 2]  #[1, 2, 3, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 17, 19, 20, 22, 23, 24, 25]
    tests = ["T1", "T2"]
    g = [(i/10)+0.01 for i in range(0,31,2)]
    g = sorted(g, reverse=True)
    for group_id in Subjects.keys():
        for subject_id in Subjects[group_id]:
            J_i = np.asarray([[0.0 for i in range(68)] for j in range(len(g))])
            PCorr = [0.0 for i in range(len(g))]
            make_input(group_id, subject_id)
            for i in range(len(g)):
                print("Analysis of Subject: ", group_id + " " + str(subject_id))
                alterGlobalCoupling(g[i])
                executeC()
                PCorr[i] = getCorrelation(group_id, subject_id)
                print("Global Coupling: ", g[i], " and Correlation: ", PCorr[i])
                (J_i)[i] = np.loadtxt("J_i.txt")
                print(J_i[i])
            try : os.remove(get_path(group_id, subject_id, tests[0]) + "/Output/J_i.txt")
            except : pass
            try : os.remove(get_path(group_id, subject_id, tests[0]) + "/Output/PCorr.txt")
            except : pass
            if not os.path.isdir(get_path(group_id, subject_id, tests[0]) + "/Output"):
                os.mkdir(get_path(group_id, subject_id, tests[0]) + "/Output")
            np.savetxt(get_path(group_id, subject_id, tests[0]) + "/Output/J_i.txt", (J_i).T, delimiter = " ")
            np.savetxt(get_path(group_id, subject_id, tests[0]) + "/Output/PCorr.txt", np.asarray(PCorr), delimiter = " ")
    os.remove("fMRI.txt")
    os.remove("J_i.txt")
    os.remove("tvbii")

Analysis of Subject:  CON 1
Global Coupling:  3.01  and Correlation:  0.3434887530759078
[1.65675 1.08811 1.93995 2.2604  1.08089 2.16556 3.06718 2.26142 1.15907
 3.26338 1.86652 1.8239  1.46171 2.6058  1.07164 1.74827 1.76948 1.42145
 1.83446 2.24355 1.95006 1.12523 2.7633  1.74286 1.10059 2.9108  3.15025
 4.03524 1.98667 2.38644 1.08902 1.59535 1.11091 1.5826  1.97164 1.18559
 1.98621 2.26372 1.17998 2.07722 5.19384 2.95913 1.10293 3.59398 2.07775
 1.46266 1.49866 3.47742 1.0957  1.4492  1.94219 2.1973  2.49287 2.06463
 2.37152 1.21184 3.07863 2.36015 1.19432 3.40874 2.98848 4.57603 1.91466
 2.36527 1.12638 1.38016 1.11775 1.77155]
Analysis of Subject:  CON 1
Global Coupling:  2.8099999999999996  and Correlation:  0.34298481377865964
[1.60487 1.08321 1.86691 2.16329 1.07667 2.0749  2.91004 2.16303 1.14658
 3.09031 1.79836 1.75827 1.42536 2.48045 1.06816 1.68916 1.70891 1.3877
 1.76904 2.14643 1.87573 1.11641 2.62691 1.68465 1.094   2.76384 2.98505
 3.80223 1.90907 2.27933 1.08412 1.5

Global Coupling:  0.41000000000000003  and Correlation:  0.30446594625760454
[1.06934 1.03183 1.08398 1.11111 1.03155 1.10077 1.18068 1.1078  1.03148
 1.20229 1.08323 1.07248 1.05503 1.1368  1.03019 1.0745  1.07528 1.04757
 1.0794  1.10904 1.08709 1.03322 1.15321 1.07444 1.0288  1.16598 1.19101
 1.27087 1.08551 1.12612 1.03182 1.06137 1.03237 1.0601  1.08857 1.03206
 1.08864 1.10937 1.03696 1.10407 1.39512 1.17047 1.02603 1.23492 1.09561
 1.05048 1.05847 1.22463 1.03075 1.05323 1.08667 1.10234 1.12478 1.09284
 1.12047 1.03567 1.18405 1.11803 1.0364  1.20983 1.16508 1.3291  1.08387
 1.11609 1.03405 1.05053 1.03069 1.07666]
Analysis of Subject:  CON 1
Global Coupling:  0.21000000000000002  and Correlation:  0.1405848713919481
[1.04573 1.02859 1.04843 1.0585  1.02865 1.05501 1.0853  1.05861 1.02509
 1.09652 1.05068 1.04225 1.03749 1.07101 1.02769 1.0459  1.04722 1.03294
 1.04832 1.05952 1.05053 1.02828 1.07831 1.04442 1.02492 1.0819  1.09184
 1.12198 1.04779 1.06878 1.02855 1.03976 1.0283

Global Coupling:  1.01  and Correlation:  0.296226122395726
[1.20573 1.0465  1.20219 1.39726 1.02597 1.3645  1.57711 1.50005 1.12126
 1.90667 1.14404 1.1932  1.09941 1.5726  1.02587 1.0848  1.2685  1.10117
 1.16189 1.30016 1.25089 1.06184 1.44264 1.26362 1.05585 1.3051  1.38547
 1.72021 1.42244 1.37394 1.03699 1.13451 1.07098 1.17926 1.14885 1.04064
 1.20548 1.30104 1.0693  1.43664 1.63629 1.46139 1.05557 1.9232  1.2169
 1.08112 1.10299 1.56112 1.07747 1.05731 1.12808 1.17443 1.20112 1.11345
 1.2748  1.04894 1.44101 1.20127 1.03837 1.34918 1.33255 1.66992 1.3726
 1.44744 1.04546 1.10664 1.05575 1.1206 ]
Analysis of Subject:  CON 2
Global Coupling:  0.81  and Correlation:  0.28271838329455945
[1.15711 1.04163 1.1519  1.30064 1.02596 1.27559 1.43861 1.37824 1.09459
 1.69504 1.11259 1.14572 1.08015 1.43357 1.02576 1.06976 1.20126 1.08
 1.12414 1.22714 1.18892 1.05281 1.33249 1.19887 1.04765 1.22822 1.28887
 1.54941 1.31697 1.28235 1.03437 1.10423 1.05948 1.13635 1.11412 1.03549
 1.154   1

Global Coupling:  1.61  and Correlation:  0.18247429713281316
[1.1814  1.18118 1.87941 1.64012 1.07437 1.6114  1.99052 1.69536 1.1814
 2.17213 1.32299 1.37164 1.24316 1.66643 1.04511 1.44405 1.49345 1.17138
 1.29663 1.716   1.49222 1.35722 2.49114 1.52034 1.09744 1.65041 2.22134
 2.2226  1.52314 1.62472 1.05916 1.3221  1.10427 1.25435 1.23477 1.20571
 1.5755  2.25768 1.06816 1.58124 2.50144 2.19091 1.11832 3.07327 1.46059
 1.36647 1.24031 2.25166 1.03751 1.16077 1.54666 1.45013 1.38878 1.97909
 1.8684  1.16454 2.42793 1.39654 1.14764 1.85457 2.32333 2.61233 1.75756
 1.98295 1.09073 1.64682 1.07165 1.5602 ]
Analysis of Subject:  PAT 1
Global Coupling:  1.41  and Correlation:  0.18654260563167088
[1.15611 1.15589 1.75524 1.54838 1.06733 1.52284 1.85151 1.59388 1.15462
 2.00701 1.27334 1.31565 1.20619 1.56926 1.04246 1.38028 1.42094 1.14583
 1.25223 1.61256 1.42134 1.30463 2.28143 1.44441 1.0853  1.55686 2.05005
 2.05025 1.44533 1.53595 1.05429 1.27346 1.09168 1.21601 1.2006  1.17504
 1.4

Global Coupling:  2.21  and Correlation:  0.33891387412409185
[1.47296 1.06549 1.93782 1.57704 1.06807 1.94865 2.42413 1.92005 1.15337
 2.77582 1.57361 1.45496 1.43254 2.12547 1.10564 1.40002 1.60535 1.46937
 1.48807 1.51072 1.80199 1.0963  2.32876 1.59314 1.11803 2.21307 2.94396
 2.58986 1.72202 1.9215  1.12507 1.3172  1.05582 1.64099 1.70809 1.09898
 1.72371 1.75839 1.07132 1.82108 3.15139 2.10575 1.13345 3.26967 1.44689
 1.57222 1.23374 2.55773 1.0881  1.58519 1.48237 1.52491 1.64012 1.8556
 1.98354 1.08955 2.43555 1.7873  1.13557 1.83589 2.76952 3.0003  2.29255
 1.92332 1.08123 1.33522 1.07981 1.7732 ]
Analysis of Subject:  PAT 2
Global Coupling:  2.01  and Correlation:  0.33665522972358497
[1.42302 1.06153 1.84208 1.51669 1.06377 1.85218 2.27963 1.8246  1.13768
 2.59313 1.5126  1.40669 1.3869  2.00902 1.09704 1.35772 1.54171 1.41886
 1.43625 1.45716 1.71837 1.08857 2.19254 1.5316  1.10697 2.08952 2.746
 2.42729 1.64621 1.82642 1.11384 1.28307 1.05293 1.57319 1.63422 1.09009
 1.649